#### FASE 2. Extracción de los datos de cada series y elaboración del DataFrame.

Primero importo las librerías que voy a usar, e instalo el Driver

In [ ]:
#Para interactuar con la web, a través del navegador Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

#Para controlar que no ejecute un click hasta que la página se haya cargado
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Para manejar errores, cuando no encuentra un elemento
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

#Para ir retrasando el web scraping y simular ser una persona
import time

#Para poder guardar mis Datos en un DataFrame
import pandas as pd

#Para guardar valores nulos (np.nan)
import numpy as np

#Instalo el webdriver de Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)

Después, defino una función para que vaya entrando en cada una de las URL extraídas en la fase anterior, y vaya tomando datos de cada serie. Esto lo hará por fragmentos (para facilitar la recogida de datos en caso de posibles errores), establecidos por otra función, y me devolverá un DataFrame de cada uno de esos fragmentos, de los cuales también hará copia en local.

In [16]:
#Defino una función para entrar en la URL de cada serie y tomar datos de ellas

def get_datos_series(df_links, start, stop):

    lista_datos_series = []

    #Para facilitar la recogida de datos (y que en caso de errores no haya que comenzar de nuevo),
    #el acceso a los URL del DataFrame será por fragmentos (delimitados por un start y un stop)
    #Estos últimos se relacionan con otra función que establece el valor de cada uno de ellos

    for url in df_links.iloc[start:stop, 1]:

        diccionario_datos_series = {}

        #Que intente entrar en cada URL, si no puede porque se haya caído, que se la salte
        try:
            driver.get(url)
        except:
            pass

        #Acepto las cookies en caso de haberlas
        try:
            WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-testid = "accept-button"]'))).click()
        except:
            pass

        #Empiezo a sacar datos y a guardarlos en un diccionario, a modo de clave-valor
        
        #Saco el id a partir de la URL (su formato es https://www.imdb.com/title/tt7165904/)
        try:
            diccionario_datos_series['ID'] = url.split(sep = "/")[4]
        except:
            diccionario_datos_series['ID'] = np.nan


        #Título (le pido que en caso de encontrar una coma, la sustituya por punto, para que no me dé problemas el csv)
        try:
            cuadro_superior = driver.find_element(By.CSS_SELECTOR, 'section > div.sc-e226b0e3-3.dwkouE > div.sc-69e49b85-0.jqlHBQ')
            diccionario_datos_series['Título'] = cuadro_superior.find_element(By.TAG_NAME, 'h1').text.replace(",", ".")
        except:
            diccionario_datos_series['Título'] = np.nan


        #Categoría (serie o miniserie) y calificación por edad
        try:    
            cuadro_superior_2 = cuadro_superior.find_elements(By.CSS_SELECTOR, 'ul.ipc-inline-list li')
            diccionario_datos_series['Categoría de serie'] = cuadro_superior_2[0].text.replace(",", ".")
        except:
            diccionario_datos_series['Categoría de serie'] = np.nan
        
        try:
            diccionario_datos_series['Calificación por edad'] = cuadro_superior_2[2].text.replace(",", ".")
        except:
            diccionario_datos_series['Calificación por edad'] = np.nan
        

        #Años

        #Obtengo el año en que se emitió por primera vez 
        #(el formato es tipo "1997-2001", por lo que solo quiero los 4 primeros caracteres)
        try:
            anio_lanzamiento = int(cuadro_superior_2[1].text[:4])
            diccionario_datos_series['Año de lanzamiento'] = anio_lanzamiento
        except:
            diccionario_datos_series['Año de lanzamiento'] = np.nan


        #Obtengo el año en que se dejó de emitir (a partir del caracter 5):
        try:

            #Si no lo pone porque aún no se ha terminado (p.ej. "2001-") 
            #pongo 2024 para poder operar con ello
            if cuadro_superior_2[1].text[4:].strip() == "–":
                anio_finalizacion = 2024
                diccionario_datos_series['Año de finalización'] = anio_finalizacion

            #Si sí lo pone porque ya se ha terminado (p.ej. el 2001 en "2001-2003"), tomo ese año    
            else:
                anio_finalizacion = int(cuadro_superior_2[1].text[5:])
                diccionario_datos_series['Año de finalización'] = anio_finalizacion

        #Si solo aparece un año (p.ej. "2001", si la serie se lanzó y terminó en el mismo año), tomo ese año
        except (ValueError):
            try:
                anio_finalizacion = int(cuadro_superior_2[1].text[:4])
                diccionario_datos_series['Año de finalización'] = anio_finalizacion
            except:
                diccionario_datos_series['Año de finalización'] = np.nan
                
        #Si no aparece el año, ponme nan        
        except (NoSuchElementException, IndexError, StaleElementReferenceException):
            diccionario_datos_series['Año de finalización'] = np.nan
            
        
        #A partir de los dos años anteriores, calculo el tiempo que la serie ha estado en emisión
        try:
            diccionario_datos_series['Años en emisión'] = anio_finalizacion - anio_lanzamiento
        except:
            diccionario_datos_series['Años en emisión'] = np.nan


        #Número de episodios
        try:
            diccionario_datos_series['Número de episodios'] = driver.find_element(By.CSS_SELECTOR, 'h3.ipc-title__text span.ipc-title__subtext').text
        except:
            diccionario_datos_series['Número de episodios'] = np.nan
            

        #Duración de cada episodio 
        try:    
            diccionario_datos_series['Duración'] = cuadro_superior_2[3].text
        except:
            diccionario_datos_series['Duración'] = np.nan


        #Puntuación en IMDb
        try:
            votos = driver.find_element(By.CSS_SELECTOR, 'div.sc-acdbf0f3-3.eWQwwe > div.sc-bde20123-0.dLwiNw')
            diccionario_datos_series['Puntuación en IMDb'] = votos.find_element(By.CLASS_NAME, 'sc-bde20123-1.cMEQkK').text.replace(",", ".")
        except:                                                                          #si encuentra una coma, que la sustituya por un punto
            diccionario_datos_series['Puntuación en IMDb'] = np.nan
        
        #Número de votos
        try:
            diccionario_datos_series['Número de votos'] = votos.find_element(By.CLASS_NAME, 'sc-bde20123-3.gPVQxL').text.replace(",", ".")
        except:                                                                      #si encuentra una coma, que la sustituya por un punto
            diccionario_datos_series['Número de votos'] = np.nan


        #Los cuatro primeros géneros de la serie
        for i in range(4): #Voy a tomar como máximo los 4 primeros géneros que me aparezcan en cada serie
            try:
                generos = driver.find_elements(By.CSS_SELECTOR, 'section.sc-69e49b85-4.ktjuZl div.ipc-chip-list__scroller a span')
                diccionario_datos_series[f'Género {i+1}'] = generos[i].text

            except: #Si los géneros no están en la parte de la página donde suelen estar, miramos en otra parte
                try:             #(ya que he visto que pueden aparecer en dos sitios diferentes de la página)
                    generos = driver.find_elements(By.CSS_SELECTOR, 'section.sc-69e49b85-4.icBxAK div.ipc-chip-list__scroller a span')
                    diccionario_datos_series[f'Género {i+1}'] = generos[i].text        
                except:
                    diccionario_datos_series[f'Género {i+1}'] = np.nan


        #Finalmente añadimos al diccionario el parámetro URL
        diccionario_datos_series['url'] = url

        #Guardo el diccionario correspondiente a cada serie en una lista de diccionarios
        lista_datos_series.append(diccionario_datos_series)
    
        #Le pido que se detenga un segundo antes de entrar en la siguiente URL, 
        #para evitar que dé error por ir demasiado rápido
        time.sleep(1)


    #Creo el DataFrame de las URL recorridas (del fragmento analizado de mi DataFrame de URL)
    df_todas_mis_series = pd.DataFrame(data = lista_datos_series, columns = diccionario_datos_series.keys())


    #Hago copia de dicho DataFrame en local, a modo de copia de seguridad
    df_todas_mis_series.to_csv(fr'df_datos_{start}_{stop-1}.csv')

    #La función me develve el DataFrame
    return df_todas_mis_series


Cargo mi DataFrame con las URL de las series, obtenido en la fase anterior

In [ ]:
df_links_completo = pd.read_csv(r'df_links_GB_completo.csv')

Defino los fragmentos de la URL que se irán utilizando para llamar a la función anterior. Esto lo hago modificando los parámetros start y stop con ayuda de un bucle while

In [1]:
#Defino dónde comenzará y terminará cada fragmento a recorrer del DataFrame de URL
start = 0
stop = 200

lista_trozos_df_datos = []

while start < 1800:    #1800 porque el len(df_links_completo) = 1707

    #Llamo a la función para cada fragmento concreto
    trozo = get_datos_series(df_links_completo, start, stop)

    #Guardo cada DataFrame resultante en una lista
    lista_trozos_df_datos.append(trozo)
    
    #Al terminar de recorrer todas las URL de cada fragmento (200 series por cada uno),
    #paso al fragmento siguiente

    start += 200
    stop += 200

Monto mi DataFrame completo con todos los datos

In [ ]:
#Concateno los DataFrame de mi lista
df_datos_completo = pd.concat(lista_trozos_df_datos, ignore_index = True)

#Hago copia en local del DataFrame completo con todos los datos
df_datos_completo.to_csv(r'mi_df_original.csv')

OPCIÓN ALTERNATIVA: En caso de que mi recogida de datos se haya detenido por algún motivo (p. ej. porque la conexión a internet al entrar a una página concreta se haya demorado más de lo previsto), se pueden modificar manualmente los valores de start y stop del bucle while anterior, para seguir tomando los datos a partir del último DataFrame cuya copia tengo guardada en local (evitando tener que comenzar desde el principio). Estos DataFrames de segmentos que tengo guardados en local se pueden concatenar a su vez por medio de otro bucle while, que se detalla a continuación:

In [2]:
#Concateno todos mis dataframes parciales (de fragmentos) guardados por la función

start = 0

mi_df_original = pd.DataFrame()

while start < 1800:

    #Como he nombrado a todos mis DataFrame guardados en local en csv de la misma forma, 
    #pero cambiando el start y el stop, puedo ir importándolos con un buble, si voy cambiando dichos números

    df_csv_concatenar = pd.read_csv(f'df_datos_{start}_{start+199}.csv')

    #Cada DataFrame importado lo concateno al anterior
    mi_df_original = pd.concat([mi_df_original, df_csv_concatenar], ignore_index=True)

    start += 200

#Hago copia en local del DataFrame completo con todos los datos
mi_df_original.to_csv(r'mi_df_original.csv')